#트리의 앙상블
###IT융합자율학부 201814076 권혁준

##정형 데이터와 비정형 데이터
- 정형 데이터
 - 어떤 구조로 구조화된 데이터
  - 생선 데이터, 와인 데이터, 상품 데이터 등
 - csv나 데이터베이스, 혹은 엑셀로 저장하여 처리하기 쉬움
 -앞서 배운 머신러닝 알고리즘을 적용하기에 알맞음
- 비정형 데이터
 - 엑셀이나 데이터베이스로 표현하기 어려운 데이터
  - 텍스트 데이터, 사진 데이터, 음악 데이터 등
 - 규칙성을 찾기 어려워 전통적인 머신러닝 알고리즘으로는 모델을 만들기 어렵지만, 신경만 알고리즘의 발전으로 사진을 인식하고 텍스트를 이해하는 모델이 연구 개발됨

##앙상블 학습
- 더 좋은 예측 결과를 만들기 위해 여러 개의 모델을 훈련하는 학습 방법
- 일반적으로 정형 데이터에서 좋은 성능을 보임

## 랜덤 포레스트
- 여러 개의 결정 트리를 합쳐서, 높은 성능의 모델을 만드는 대표적인 앙상블 학습 모델
- 결정 트리를 랜덤하게 만들어 숲을 만들고, 각 결정 트리의 예측으로부터 최종 예측을 수행
- 각 결정 트리를 훈련 데이터로부터 랜덤하게 추출한 데이터를 사용하여 훈련
-RandomForestClassifier
 - sklerarn.ensemble 모듈
 - n_estimator=100: 결정 트리의 수
 - obb_score=False
 - n_jobs=1

##부트스트랩
- 데이터 세트에서 중복을 허용하여 데이터를 샘플링하는 방식(복원추출)
- 부트스트랩 샘플
 - 부트스트랩으로 샘플링한 데이터
 - 일반적으로 훈련 세트의 크기와 같은 수를 샘플링
- 훈련 세트에서 내용이 다른 여러 개의 훈련 데이터를 생성하여 각 결정 트리가 다른 훈련 세트로 학습할 수 있도록 함

## 특징 선택
- 훈련 세트에서 일부 특징만을 무작위로 고른 다음 이 중에서 최선의 분할을 찾음
 - 분류 모델인 RAndomForestClassifier는 전체 특성 개수의 제곱근만큼 특성을 선택하여 훈련하고, 각 트리의 클래스별 확률을 평균하여 가장 높은 확률을 가진 클래스를 예측으로 삼음
 - 회귀 모델인 RandomForestRegressor는 전체 특성을 사용하되, 각 트리의 예측을 평균하여 예측으로 삼음

In [23]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol', 'sugar', 'pH']].to_numpy() # to.numpy()로 2차원 배열로 만들어줌
target = wine['class'].to_numpy()
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

##랜덤 포레스트에서의 부트스트랩과 특징 선택
- 랜덤하게 선택한 샘플과 특성의 사용으로 훈련 세트의 과대적합을 방지하고 테스트 세트에서 일정 이상의 성능을 획득

In [24]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs=-1, random_state=42)

In [25]:
scores = cross_validate(rf, train_input, train_target,
                        return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


In [26]:
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


##특성 중요도
- 결정 트리 모델로부터 얻은 특성 중요도와 비교
 - 도수와 ph의 중요도는 높아지고, 당도의 중요도가낮아짐
 - 랜덤 포레스트 모델이 특성의 일부를 무작위로 선택하여 훈련하여, 하나의 특성에 과도하게 집중하지 않고, 좀 더 많은 특성이 훈련에 참여할 기회를 얻음

##OOB샘플
- 훈련 세트의 샘플 중 부트스트랩 샘플에 포함 되지 않은 남은 샘플
- 검증 세트와 같이 부트스트랩 샘플로 훈련한 결정 트리 모델의 성능 평가에 사용 가능
 - 교차 검증을 대신할 수 있어 모델 훈련에 더 많은 훈련 데이터 사용이 가능

In [27]:
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


##엑스트라 트리즈
- 랜덤 포레스트와 비슷한 앙상블 학습 모델
- 부트스트랩을 사용하지 않음
- 엑스트라 트리
 - DecisionTreeClassifier 클래스의 매개변수 spliter='best'를 'random' 으로 설정하면 정보이득을 고려하지 않고 무작위로 노드를 분할
 - 랜덤으로 노드를 분할하기 때문에 빠른 학습이 장점
-ExtraTreesClassifier 클래스
 - sklearn.ensemble 모듈

In [28]:
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target,
                        return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [29]:
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


##그레디언트 부스팅
- 깊이가 얕은 결정트리를 사용하여 이전 트리의 오차를 보완하는 방식의 앙상블 학습 모델
- 경사하강법을 이용하여 트리를 앙상블에 추가함
 - 분류: 로지스틱 함수
 - 회귀: 평균 제곱 오차 함수
- 결정 트리의 깊이가 얕아 과대적합에 강하고, 높은 일반화 성능 기대 가능
- 일반적으로 랜덤 포레스트보다 좋은 성능을 보이나, 트리를 추가하는 방식으로 훈련 속도가 느림
- GradientBoostingClassifier 클래스
 - sklearn.ensemble 모듈
 - n_estimators=100
 - learning_rate=0.1 학습률
 - subsample=1.0: 결정트리 훈련에 사용할 훈련 셑의 비율 (1보다 작으면 일부)

In [30]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target,
                        return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


In [31]:
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
scores = cross_validate(gb, train_input, train_target,
                        return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [32]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15872278 0.68010884 0.16116839]


##히스토그램 기반 그레이디언트 부스팅

In [33]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target,
                        return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


In [34]:
from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target,
                                n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


In [35]:
result = permutation_importance(hgb, test_input, test_target,
                                n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


In [36]:
hgb.score(test_input, test_target)

0.8723076923076923

In [37]:
from xgboost import XGBClassifier
xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(hgb, train_input, train_target,
                        return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


In [38]:
from lightgbm import LGBMClassifier
lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target,
                        return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

[LightGBM] [Info] Number of positive: 3151, number of negative: 1006
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000239 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 372
[LightGBM] [Info] Number of data points in the train set: 4157, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.757999 -> initscore=1.141738
[LightGBM] [Info] Start training from score 1.141738
[LightGBM] [Info] Number of positive: 3151, number of negative: 1006
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000151 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 370
[LightGBM] [Info] Number of data points in the train set: 4157, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.757999 -> initscore=1.141738
[LightGBM] [Info] Start training from score 1.141738
[LightGBM] [Info] Nu